In [1]:
import seaborn as sns
import sklearn as sk 
import pandas as pd

In [78]:
df = pd.read_csv('simpsons_dataset.csv').dropna()
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [3]:
df['raw_character_text'].value_counts().head()

Homer Simpson          27850
Marge Simpson          13172
Bart Simpson           12995
Lisa Simpson           10756
C. Montgomery Burns     3077
Name: raw_character_text, dtype: int64

Bart has more lines than Lisa????

In [4]:
df_subset = df[(df['raw_character_text']=='Bart Simpson') | (df['raw_character_text'] == 'Lisa Simpson')]
# df.loc[(df["""])] kan ook. 
df_subset.head()

,raw_character_text,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
7,Bart Simpson,Victory party under the slide!
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
11,Lisa Simpson,Do you know where I could find him?


In [5]:
from sklearn.feature_extraction.text import CountVectorizer #object you need, you take the model?

#convert it to unicode (standard for text that is used worldwide.) 
# wouldn't work in the CountVectorizer without it
text = df_subset['spoken_words'].values.astype('U')

# Counts all the words in the document 
# It will take out the very frequent words like 'I', 'the', etc. 
# these words are uninformative
# the empty model
vect = CountVectorizer(stop_words='english')
# we create the count of all the words in the text
# fitting the model
vect = vect.fit(text) 
# words that are in the documents are added to a variable.
feature_names = vect.get_feature_names() 
# shows the amount of unique words in the dataset + a sample of the words
print(f"There are {len(feature_names)} words in the vocabulary. A selection: {feature_names[500:520]}")

There are 14257 words in the vocabulary. A selection: ['anguished', 'angus', 'anima', 'animal', 'animals', 'animated', 'animation', 'animators', 'anka', 'ankle', 'ann', 'annapolis', 'anne', 'annie', 'anniversary', 'annnnd', 'announce', 'announcement', 'announcements', 'announcer']


In [6]:
matrix = vect.transform(text) 
print(matrix[0:500,0:500]) 

  (23, 424)	1
  (38, 325)	1
  (43, 266)	1
  (61, 269)	1
  (72, 356)	1
  (78, 264)	1
  (80, 304)	1
  (96, 192)	1
  (98, 396)	1
  (149, 328)	1
  (154, 325)	1
  (155, 451)	1
  (161, 325)	1
  (162, 325)	1
  (184, 461)	1
  (205, 325)	1
  (208, 397)	1
  (229, 270)	1
  (235, 404)	1
  (256, 325)	1
  (284, 325)	1
  (291, 493)	1
  (292, 163)	1
  (315, 300)	1
  (318, 281)	1
  (353, 450)	1
  (355, 397)	1
  (359, 449)	1
  (363, 24)	1
  (363, 449)	1
  (381, 129)	1
  (382, 325)	1
  (383, 70)	1
  (389, 38)	1
  (389, 91)	1
  (391, 446)	1
  (393, 126)	1
  (405, 52)	1
  (405, 319)	1
  (405, 343)	1
  (408, 449)	1
  (414, 196)	1
  (422, 360)	1
  (457, 304)	1


In [7]:
df_sub = pd.DataFrame(matrix.toarray()) 
df_sub.index = df_subset['raw_character_text'] 
df_sub.columns = feature_names

In [8]:
df_sub.iloc[0:4, 1000:1015]

,bartholemew,bartholomew,bartish,bartman,barto,bartrand,bartron,barts,barty,bas,base,baseball,based,basement,basements
raw_character_text,,,,,,,,,,,,,,,
Lisa Simpson,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Lisa Simpson,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bart Simpson,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Lisa Simpson,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Start of exercise 2

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

y = df_subset['raw_character_text']
X = df_sub.loc[:]
#.loc[:] is niet nodig. 

In [10]:
# training data and test data
X_train_k, X_test_k, y_train_k, y_test_k = train_test_split(X, y, test_size=0.3)

In [11]:
# we create the model, create an object (box, called nb 
nb = MultinomialNB()
nb.fit(X_train_k, y_train_k)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [39]:
# using the object
# nb. the . means it's either an object or a method.
# because we need parentheses it's a method. 
# evaluating the model
y_pred = nb.predict(X_test_k)
y_pred

array(['Bart Simpson', 'Lisa Simpson', 'Lisa Simpson', ...,
       'Lisa Simpson', 'Bart Simpson', 'Bart Simpson'], dtype='<U12')

In [13]:
accuracy = nb.score(X_test_k, y_test_k)
print(f' The accuracy is {accuracy}.')

 The accuracy is 0.6389278697726635.


The accuracy is quite low, at 63,8%.

### Exercise 3

In [31]:
from sklearn.metrics import confusion_matrix

In [32]:
cm = confusion_matrix(y_test_k, y_pred)
cm

array([[3018,  949],
       [1624, 1535]])

In [33]:
conf_matrix = pd.DataFrame(cm, index=['Bart Simpson(actual)', 'Lisa Simpson(actual)'], 
                           columns = ['Bart Simpson(pred)', 'Lisa Simpson(pred)']) 
conf_matrix

,Bart Simpson(pred),Lisa Simpson(pred)
Bart Simpson(actual),3018,949
Lisa Simpson(actual),1624,1535


In [34]:
nb.classes_

array(['Bart Simpson', 'Lisa Simpson'], dtype='<U12')

In [35]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test_k, y_pred)
print(f'The accuracy is {accuracy}.')

The accuracy is 0.6389278697726635.


#### Bart
\begin{equation}
\frac{3019}{(3019+894)} = 0.7715
\end{equation}

\begin{equation*}
\frac{3019}{(3019+1662)} = 0.6449
\end{equation*}

These equations show how the recall is 0.7715<br>
And the precision is 0.6449

#### Lisa

\begin{equation}
\frac{1551}{(1551+1662)} = 0.4827
\end{equation}

\begin{equation*}
\frac{1551}{(1551+894)} = 0.6344
\end{equation*}

These equations show how the recall is 0.4827<br>
And the precision is 0.6344

In [51]:
# easier way to do this.
from sklearn.metrics import classification_report
print(classification_report(y_test_k,y_pred, nb.classes_))

              precision    recall  f1-score   support

Bart Simpson       0.65      0.76      0.70      3967
Lisa Simpson       0.62      0.49      0.54      3159

 avg / total       0.64      0.64      0.63      7126



In [66]:
proba = nb.predict_proba(X)
proba = pd.DataFrame(proba)
proba.columns = ("Bart", "Lisa")
proba.head()

#print(df.iloc[0,1])
# print(nb.predict_proba(X[0:4]))

,Bart,Lisa
0,0.035228,0.964772
1,0.633464,0.366536
2,0.495146,0.504854
3,0.001121,0.998879
4,0.544615,0.455385


In [70]:
index = 0
predictions = []
for line in df_subset["spoken_words"]:
    if index < 100:
        if proba["Bart"][index] > 0.5:
            test = {
                "Predicted character" : "Bart Simpson",
                "spoken_words" : line
            }
            predictions.append(test)
            
        elif proba["Bart"][index] < 0.5:
            test = {
                "Predicted character" : "Lisa Simpson",
                "spoken_words" : line
            }
            predictions.append(test)
            
    index = index+1


In [79]:
predictions = pd.DataFrame(predictions)
result = pd.merge(predictions, df_subset, on="spoken_words")
result = result[['raw_character_text','Predicted character','spoken_words']]
result.columns = ['Character', 'Predicted character', 'Line']
result.head(10)

,Character,Predicted character,Line
0,Lisa Simpson,Lisa Simpson,Where's Mr. Bergstrom?
1,Lisa Simpson,Bart Simpson,That life is worth living.
2,Bart Simpson,Lisa Simpson,Victory party under the slide!
3,Lisa Simpson,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
4,Lisa Simpson,Bart Simpson,Do you know where I could find him?
5,Lisa Simpson,Lisa Simpson,"The train, how like him... traditional, yet en..."
6,Lisa Simpson,Lisa Simpson,"I see he touched you, too."
7,Bart Simpson,Bart Simpson,"Hey, thanks for your vote, man."
8,Bart Simpson,Bart Simpson,"Well, you got that right. Thanks for your vote..."
9,Bart Simpson,Lisa Simpson,"Well, don't sweat it. Just so long as a couple..."


In [83]:
# Jonas' solution:

# for i in range(10):
#     prob = nb.predict_proba(X[i])
#     print(f"line: {i}. {df_subset.iloc[i,1]}")
#     print(f"Bart: {prob[0,0]}, Lisa: {prob[0,1]}")